In [20]:
from __future__ import division, print_function, absolute_import

import csv
import cv2
import os
import tensorflow as tf

import shutil
import urllib.request
import tarfile
print('general imports done')
print('tensorflow version' + tf.__version__)


general imports done
tensorflow version1.15.0


In [21]:
#checks if the images box position is placed within the image.

#note: while this doesn't checks if the boxes/annotatoins are correctly
# placed around the object, Tensorflow will through an error if this occured.

# loops over both train_labels and test_labels csv files to do the check
# returns the image name where an error is found
# return the incorrect attributes; xmin, ymin, xmax, ymax.
# if found, it should be deleted, both image and the line in the csv file

images_path = 'gun_detection/data/images'
for CSV_FILE in ['gun_detection/data/train_labels.csv', 'gun_detection/data/test_labels.csv']:
  with open(CSV_FILE, 'r') as fid:
      print('[*] Checking file:', CSV_FILE)
      file = csv.reader(fid, delimiter=',')
      first = True
      cnt = 0
      error_cnt = 0
      error = False
      for row in file:
          if error == True:
              error_cnt += 1
              error = False
          if first == True:
              first = False
              continue
          cnt += 1
          name, width, height, xmin, ymin, xmax, ymax = row[0], int(row[1]), int(row[2]), int(row[4]), int(row[5]), int(row[6]), int(row[7])
          path = os.path.join(images_path, name)
          img = cv2.imread(path)
          if type(img) == type(None):
              error = True
              print('Could not read image', img)
              continue
          org_height, org_width = img.shape[:2]
          if org_width != width:
              error = True
              print('Width mismatch for image: ', name, width, '!=', org_width)
          if org_height != height:
              error = True
              print('Height mismatch for image: ', name, height, '!=', org_height)
          if xmin > org_width:
              error = True
              print('XMIN > org_width for file', name)
          if xmax > org_width:
              error = True
              print('XMAX > org_width for file', name)
          if ymin > org_height:
              error = True
              print('YMIN > org_height for file', name)
          if ymax > org_height:
              error = True
              print('YMAX > org_height for file', name)
          if error == True:
              print('Error for file: %s' % name)
              print()
      print()
      print('Checked %d files and realized %d errors' % (cnt, error_cnt))
      print("-----")


[*] Checking file: gun_detection/data/train_labels.csv

Checked 2776 files and realized 0 errors
-----
[*] Checking file: gun_detection/data/test_labels.csv
XMIN > org_width for file armas (2815).jpg
XMAX > org_width for file armas (2815).jpg
YMIN > org_height for file armas (2815).jpg
YMAX > org_height for file armas (2815).jpg
Error for file: armas (2815).jpg


Checked 688 files and realized 1 errors
-----


In [22]:
# Some models to train on
MODELS_CONFIG = {
    'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_coco_2018_03_29',
        'pipeline_file': 'ssd_mobilenet_v2_coco.config',
    },
    'faster_rcnn_inception_v2': {
        'model_name': 'faster_rcnn_inception_v2_coco_2018_01_28',
        'pipeline_file': 'faster_rcnn_inception_v2_pets.config',
    },
    'rfcn_resnet101': {
        'model_name': 'rfcn_resnet101_coco_2018_01_28',
        'pipeline_file': 'rfcn_resnet101_pets.config',
    }
}

# I chose ssd_mobilenet_v2 for this project, you could choose any
selected_model = 'ssd_mobilenet_v2'
print('pre-trained model seleted:' + selected_model)


pre-trained model seleted:ssd_mobilenet_v2


In [23]:
# testing the model builder in folder gun_detection/models/research
!python3 gun_detection/models/research/object_detection/builders/model_builder_test.py
print('mode builder tested')

mode builder tested


In [24]:
# 1. Based on the model selecting at the top of this notebook, downloading the model selected and extracting its content.
# 2. Creating a dir to save the model while training.

In [25]:
# Name of the object detection model to use.
MODEL = MODELS_CONFIG[selected_model]['model_name']

# Name of the pipline file in tensorflow object detection API.
pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']

#selecting the model
MODEL_FILE = MODEL + '.tar.gz'

#creating the downlaod link for the model selected
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

#the distination folder where the model will be saved
fine_tune_dir = 'gun_detection/models/research/pretrained_model'

#checks if the model has already been downloaded
if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

#unzipping the file and extracting its content
tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

# creating an output file to save the model while training
os.remove(MODEL_FILE)
if (os.path.exists(fine_tune_dir)):
    shutil.rmtree(fine_tune_dir)
os.rename(MODEL, fine_tune_dir)


In [26]:
# 1. Adding the path for the TFRecords files and pbtxt,batch_size,num_steps,num_classes to the configuration file.
# 2. Adding some Image augmentation.
# 3. Creating a directory to save the model at each checkpoint while training.

#the path to the folder containing all the sample config files
CONFIG_BASE = "gun_detection/models/research/object_detection/samples/configs/"

#path to the specified model's config file
model_pipline = os.path.join(CONFIG_BASE, pipeline_file)
model_pipline


'gun_detection/models/research/object_detection/samples/configs/ssd_mobilenet_v2_coco.config'

In [29]:

!python3 gun_detection/models/research/object_detection/model_main.py \
    --pipeline_config_path={model_pipline}\
    --model_dir={model_dir} \
    --alsologtostderr \

W0828 12:27:59.296673 4533874112 model_lib.py:771] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting train_steps: None
I0828 12:27:59.296842 4533874112 config_util.py:552] Maybe overwriting train_steps: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0828 12:27:59.296900 4533874112 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1
I0828 12:27:59.296955 4533874112 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: 1
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I0828 12:27:59.297009 4533874112 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W0828 12:27:59.297103 4533874112 model_lib.py:787] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
INFO:tensorflow:create_estimator_and_inputs: use_tpu False, export_to_tpu None
I0828 12:27:59.297165 